In [6]:
!conda env list

# conda environments:
#
base                     /home/yifengz/miniconda3
dinov2                *  /home/yifengz/miniconda3/envs/dinov2
gr1-test                 /home/yifengz/miniconda3/envs/gr1-test
hamer                    /home/yifengz/miniconda3/envs/hamer
humanoid                 /home/yifengz/miniconda3/envs/humanoid
robosuite-usd            /home/yifengz/miniconda3/envs/robosuite-usd



In [7]:
!conda list

# packages in environment at /home/yifengz/miniconda3/envs/dinov2:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                 conda_forge    conda-forge
_openmp_mutex             4.5                       2_gnu    conda-forge
absl-py                   2.1.0                    pypi_0    pypi
accelerate                0.28.0                   pypi_0    pypi
addict                    2.4.0                    pypi_0    pypi
antlr4-python3-runtime    4.9.3                    pypi_0    pypi
anyio                     4.3.0                    pypi_0    pypi
argon2-cffi               23.1.0                   pypi_0    pypi
argon2-cffi-bindings      21.2.0                   pypi_0    pypi
arrow                     1.3.0                    pypi_0    pypi
asttokens                 2.4.1              pyhd8ed1ab_0    conda-forge
async-lru                 2.0.4                    pypi_0    pypi
attrs                     23.2.0                   

In [3]:
from orion.algos.human_video_oogs import HumanVideoOOGs
from orion.algos.oog import OpenWorldObjectSceneGraph
from orion.utils.misc_utils import *
from orion.utils.o3d_utils import *
from orion.utils.correspondence_utils import CorrespondenceModel
human_video_oogs = HumanVideoOOGs()
# correspondence_model = CorrespondenceModel()
# human_video_oogs.set_correspondence_model(correspondence_model)

ModuleNotFoundError: No module named 'sklearn'

In [4]:
human_video_annotation_path = "annotations/human_demo/iphone_place_chip_bag_on_plate_1_demo" # iphone_front_boat_demo
human_video_oogs.init()
human_video_oogs.generate_from_human_video(human_video_annotation_path)


NameError: name 'human_video_oogs' is not defined

In [ ]:
idx = 3

human_video_oogs.get_graph(0).draw_overlay_image(mode="object")

input_annotation = human_video_oogs.get_graph(idx).input_annotation
overlay_image = overlay_xmem_mask_on_image(
                human_video_oogs.get_graph(idx).input_image, 
                human_video_oogs.get_graph(idx).input_annotation,
                use_white_bg=False,
                rgb_alpha=0.3)



overlay_image = overlay_image * (input_annotation == 0)[..., None] + human_video_oogs.get_graph(idx).input_image * (input_annotation != 0)[..., None]
plotly_draw_image(overlay_image, width=640, height=480)


In [ ]:
human_video_annotation_path = "annotations/human_demo/iphone_front_boat_demo" # iphone_front_boat_demo
human_video_oogs.init()
human_video_oogs.generate_from_human_video(human_video_annotation_path)


### Source image and depth

In [ ]:
plotly_draw_image(human_video_oogs.get_graph(2).input_image, width=640, height=480)
plotly_draw_image(human_video_oogs.get_graph(1).input_depth, width=640, height=480)
human_video_oogs.get_graph(0).draw_overlay_image(mode="point", width=640, height=480, default_point_color="rgb(30, 122, 122)")

### Draw the scene and the objects from human video
1. Draw the scene
2. Draw the scene with objects being annotated with segmentation mask
3. Trajectories that show the 3D motion of the objects

In [ ]:
human_video_oogs.get_graph(1).draw_scene_3d(downsample=False, 
                                            depth_trunc=1.0,  
                                            use_white_bg=True,
                                            no_background=True, 
                                            overlay_annotation=True)

In [ ]:
trajs = human_video_oogs.get_graph(1).get_world_trajs(object_ids=[2])
trajs = np.delete(trajs, 34, axis=0)
human_video_oogs.get_graph(1).draw_objects_list_3d(object_ids=[1, 2], downsample=False, no_background=True)

In [ ]:
trajs = human_video_oogs.get_graph(1).get_world_trajs(object_ids=[2])
trajs = np.delete(trajs, [10, 17], axis=0)
human_video_oogs.get_graph(1).draw_scene_3d(
                                            additional_points=trajs,
                                            downsample=False, no_background=True,
                                            depth_trunc=1.0,  overlay_annotation=True, 
                                            additional_point_draw_lines=True,)

### Draw individual object point clouds

In [ ]:
human_video_oogs.get_graph(1).draw_objects_3d(object_id=2, downsample=False,  no_background=True, draw_keypoints=True, keypoints_rgb_str="(30, 200, 200)", uniform_color=True)

In [ ]:
trajs = human_video_oogs.get_graph(1).get_world_trajs(object_ids=[1])
trajs = np.delete(trajs, [10, 17], axis=0)
human_video_oogs.get_graph(1).draw_objects_3d(object_id=1, downsample=False,  no_background=True, draw_trajs=False, additional_point_draw_lines=True, keypoints_rgb_str="(30, 122, 122)", 
                                              additional_points=trajs, 
                                              uniform_color=False)

In [ ]:
import trimesh
import plotly.graph_objects as go

# Load the .obj file using trimesh
mesh = trimesh.load('annotations/human_demo/iphone_front_boat_demo/hamer_output/0039_0.obj')

# Extract vertices and faces
vertices = mesh.vertices
faces = mesh.faces

# Create a 3D plot
fig = go.Figure(data=[go.Mesh3d(x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2], 
                                i=faces[:, 0], j=faces[:, 1], k=faces[:, 2], 
                                 opacity=0.50)])

# Adjust the camera view as desired
fig.update_layout(scene_camera=dict(
    eye=dict(x=1.5, y=1.5, z=1.5)  # Example camera position
),
        scene=dict(
            xaxis=dict(showbackground=False, zeroline=False, showgrid=False, showticklabels=False, showaxeslabels=False, visible=False),
            yaxis=dict(showbackground=False, zeroline=False, showgrid=False, showticklabels=False, showaxeslabels=False, visible=False),
            zaxis=dict(showbackground=False, zeroline=False, showgrid=False, showticklabels=False, showaxeslabels=False, visible=False),
        ),
        paper_bgcolor='rgba(0,0,0,0)',  # Transparent background
        plot_bgcolor='rgba(0,0,0,0)',  # Transparent background
        margin=dict(l=0, r=0, b=0, t=0),  # No margins
        showlegend=False,
)

fig.show()